In [ ]:
import pandas as pd

df = pd.DataFrame()

In [ ]:
import os

In [ ]:
list_file = list(os.walk('./part'))[0][2]
if '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
if '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
list_file.sort()
    
len(list_file)

In [ ]:
from bs4 import BeautifulSoup
from lxml import etree

In [ ]:
a = 0

with open('./part/' + list_file[a], 'r', encoding='UTF-8') as file:
    resp = file.read()
    
soup = BeautifulSoup(resp, 'lxml')
html = etree.HTML(str(soup))

with open('./part.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

In [ ]:
title = html.xpath('//h1[@class="product__title-link"]/text()')[0].strip()

title

In [ ]:
subtitle = html.xpath('//h1[@class="product__title-link"]/span[@class="product__subtitle"]/text()')
if len(subtitle) == 0:
    subtitle = ''
else:
    subtitle = subtitle[0].strip()
    
subtitle

In [ ]:
import re

In [ ]:
artkl = re.sub('\n +', '\n', html.xpath('//div[@class="product__artkl"]/text()')[0].strip())

print(artkl)

In [ ]:
product_id = html.xpath('//div[@class="product"]/@data-article-id')[0].strip()

product_id

In [ ]:
price = html.xpath('//div[@class="product__new-price"]/text()')[0].strip()

price

In [ ]:
list_oe = []
list_li = html.xpath('//div[@class="product-oem__list"]/ul/li')
for li in list_li:
    if len(li.xpath('./a')) == 0:
        list_oe.append(re.sub('^.*?OE-', '', li.xpath('./text()')[0].strip()))
    else:
        list_oe.append(re.sub('^.*?OE-', '', li.xpath('./a/text()')[0].strip()))
        
list_oe

In [ ]:
kit = '\n'.join([re.sub('\n +', ' ', kit.strip()) for kit in html.xpath('//div[@class="product__kit"]/ul/li/text()')])

print(kit)

In [ ]:
src = html.xpath('//div[contains(@class, "main-image")]/img/@src')[0].strip()

src

In [ ]:
maker_id = ';'.join([maker.strip() for maker in html.xpath('//div[@class="compatibility__maker-title"]/@data-maker-id')])

maker_id

In [ ]:
list_tr = html.xpath('//table[@class="product__table"]/tr')

list_tr

In [ ]:
list_li = html.xpath('//div[@class="product-analogs__wrapper"]/ul/li')

list_li

In [ ]:
df_temp = pd.DataFrame([{'序号': int(list_file[a].replace('.txt', '')),
                         'Title': title,
                         'Subtitle': subtitle,
                         'Artkl': artkl,
                         'Product_Id': product_id,
                         'Price': price,
                         'Vehicle_1': '',
                         'Vehicle_2': '',
                         'OE': ';'.join(list_oe),
                         'Kit': kit,
                         'Pic': '',
                         'Url': '',
                         'Src': src,
                         'Maker_Id': maker_id}])
    
for tr in list_tr:
    name = tr.xpath('./td[1]/text()')[0].strip()
    if name[-1] == ':':
        df_temp.loc[0, name.upper()[:-1]] = tr.xpath('./td[2]/text()')[0].strip()
    else:
        df_temp.loc[0, name.upper()] = tr.xpath('./td[2]/text()')[0].strip()

for li in list_li:
    name = li.xpath('./span/text()')[0].strip()
    if name[-1] == ':':
        df_temp.loc[0, name.lower()[:-1]] = ''.join(li.xpath('./text()')).strip()
    else:
        df_temp.loc[0, name.lower()] = ''.join(li.xpath('./text()')).strip()

df = pd.concat([df, df_temp], ignore_index=True).fillna('')
    
len(df)

In [ ]:
df = df.drop_duplicates(ignore_index=True)
df = df.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
df.to_excel('./part.xlsx', index=False)

df